# Video Recording Cell - Add RAW video recording capability (CLEAN VERSION)
# !/usr/bin/python377


In [4]:
# ===== CONVERT .H5 MODEL TO INT8 QUANTIZED .TFLITE (RUN ON WINDOWS) =====
# INT8 quantization uses ~4x less memory than float32 - critical for low-RAM RPi
from tensorflow import keras
import tensorflow as tf
import numpy as np

h5_path = r"C:\Users\thaim\Downloads\detector_classifier_mobilenetv2_final.h5"
tflite_path = r"C:\Users\thaim\Downloads\detector_classifier_mobilenetv2_int8.tflite"

model = keras.models.load_model(h5_path)
print("Model loaded. Converting to INT8 TFLite...")

# Representative dataset generator for INT8 calibration (uses random images)
def representative_dataset():
    for _ in range(100):
        data = np.random.rand(1, 224, 224, 3).astype(np.float32) * 255.0  # Random RGB images
        yield [data]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.uint8   # Input: uint8 (0-255)
converter.inference_output_type = tf.float32  # Output: float32 (for sigmoid probability)

tflite_model = converter.convert()

with open(tflite_path, 'wb') as f:
    f.write(tflite_model)

print(f"Saved: {tflite_path} ({len(tflite_model) / 1024 / 1024:.2f} MB)")
print("Transfer to RPi: scp detector_classifier_mobilenetv2_int8.tflite reucam@169.254.104.28:/home/reucam/dev/fgvideo/src/MODEL_WEIGHTS/")

Model loaded. Converting to INT8 TFLite...
INFO:tensorflow:Assets written to: C:\Users\thaim\AppData\Local\Temp\tmp2ivinmta\assets


INFO:tensorflow:Assets written to: C:\Users\thaim\AppData\Local\Temp\tmp2ivinmta\assets


Saved artifact at 'C:\Users\thaim\AppData\Local\Temp\tmp2ivinmta'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_1')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  2262406910352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406912080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406912272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406911888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406910928: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406912656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406914000: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406914192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406913808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262406913040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2262

c:\Users\thaim\AppData\Local\Programs\Python\Python312\Lib\site-packages\tensorflow\lite\python\convert.py:997: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Saved: C:\Users\thaim\Downloads\detector_classifier_mobilenetv2_int8.tflite (2.74 MB)
Transfer to RPi: scp detector_classifier_mobilenetv2_int8.tflite reucam@169.254.104.28:/home/reucam/dev/fgvideo/src/MODEL_WEIGHTS/


In [ ]:
# ===== TENSORFLOW INSTALLATION COMMANDS FOR RASPBERRY PI =====
# Run these commands in the Raspberry Pi terminal (via MobaXterm SSH)
# Do NOT run this cell - these are terminal commands only
# RPi IP: 169.254.104.28 (link-local - no internet, use offline install)

"""
========== OFFLINE INSTALLATION (RPi has no internet) ==========

STEP 1: On your Windows PC (with internet), open cmd and run:
    pip download tensorflow -d C:\tf_wheels

STEP 2: Transfer to RPi (from Windows cmd or MobaXterm local terminal):
    scp -r C:\tf_wheels reucam@169.254.104.28:/home/reucam/

STEP 3: On RPi (via SSH), install from local files:
    pip3 install /home/reucam/tf_wheels/*.whl

STEP 4: Verify installation on RPi:
    python3 -c "import tensorflow as tf; print('TensorFlow:', tf.__version__)"

STEP 5: Create model directory on RPi:
    mkdir -p /home/reucam/dev/fgvideo/src/MODEL_WEIGHTS

STEP 6: Copy your model file (from Windows):
    scp detector_classifier_mobilenetv2_final.h5 reucam@169.254.104.28:/home/reucam/dev/fgvideo/src/MODEL_WEIGHTS/

========== ALTERNATIVE: TensorFlow Lite (lighter, faster) ==========
# If full TensorFlow is too heavy for RPi, use TFLite instead:
# On Windows: pip download tflite-runtime -d C:\tf_wheels
# Then transfer and install same way as above

========== IF YOU NEED DEPENDENCIES ==========
# These may already be installed. If pip install fails, try on RPi:
    sudo apt-get install -y python3-pip python3-dev
    sudo apt-get install -y libhdf5-dev libhdf5-103
    sudo apt-get install -y libatlas-base-dev
"""
print("See comments above for offline installation commands")

In [ ]:
# ===== LED DETECTOR WITH AI MODEL VERIFICATION (TFLite) =====
# Mode 1: Full FFT scan | Mode 2: Refine bbox | Mode 3: Model prediction | Mode 4: Focused FFT + validation
import struct
import sys
import threading
import logging
import time
import cv2
import numpy as np
from scipy.signal import find_peaks
import configparser
import os
import json
import math
import signal
import atexit
import fcntl

# TFLite imports (for armv7l RPi)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
TFLITE_AVAILABLE = False
tflite_interpreter = None
try:
    import tflite_runtime.interpreter as tflite
    TFLITE_AVAILABLE = True
    logging.info("TFLite runtime loaded successfully")
except ImportError:
    logging.warning("TFLite not available - Mode 3 will auto-confirm all detections")

In [ ]:
# ===== CONFIGURATION AND INITIALIZATION =====
cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(1)
signal.signal(signal.SIGPIPE, signal.SIG_DFL)

sys.path.append("/var/fgvideo/src")
sys.path.append("/var/fgvideo/src/dist-packages")

from ReuCameraPy.ReuCameraPy import *
from ReuOnvifPy import config

logging.basicConfig(level=logging.INFO, format="[%(levelname).1s] %(message)s")

FRAME_W, FRAME_H = config.get_snapshot_resolution()

# Load JSON config
config_path = os.path.join(os.path.dirname(__file__), 'camera_alignment_config.json')
try:
    with open(config_path, 'r') as f:
        cfg = json.load(f)
except FileNotFoundError:
    logging.error(f"Configuration file not found at {config_path}")
    sys.exit(1)
except json.JSONDecodeError:
    logging.error(f"Error decoding JSON from {config_path}")
    sys.exit(1)

# ===== CONFIGURABLE PARAMETERS (CHANGE THESE AS NEEDED) =====
MARGIN_MODE4 = 40                    # Margin for Mode 4 ROI (bigger box for focused scanning)
MODEL_VALIDATION_INTERVAL = 10       # Re-run model every N frames in Mode 4
MODEL_CONFIDENCE_THRESHOLD = 0.5     # Threshold for positive prediction (0.5 = 50%)
FALSE_ALARM_DISPLAY_TIME = 4.0       # Seconds to display FALSE ALARM
MODEL_INPUT_SIZE = 224               # Model input size (224x224)

# ===== LOAD TFLITE MODEL (INT8 QUANTIZED) =====
MODEL_PATH = "/home/reucam/dev/fgvideo/src/MODEL_WEIGHTS/detector_classifier_mobilenetv2_int8.tflite"
tflite_interpreter = None
input_details = None
output_details = None
if TFLITE_AVAILABLE:
    try:
        tflite_interpreter = tflite.Interpreter(model_path=MODEL_PATH)
        tflite_interpreter.allocate_tensors()
        input_details = tflite_interpreter.get_input_details()
        output_details = tflite_interpreter.get_output_details()
        logging.info(f"TFLite INT8 model loaded: {MODEL_PATH}")
    except Exception as e:
        logging.error(f"Failed to load TFLite model: {e}")
        tflite_interpreter = None

# ===== SENSOR/CAMERA PARAMETERS =====
REUCAM_NOTIFY_FIREBORDER = 0x8010
sensor_width = cfg["sensor_width"]
sensor_height = cfg["sensor_height"]
pixel_size = cfg["pixel_size"]
focal_length = cfg["focal_length"]
BOX_SIZE = cfg["BOX_SIZE"]
LED_FREQ = cfg["LED_FREQ"]
FREQ_TOL = cfg["FREQ_TOL"]
FREQ_RANGE = 2
CALC_LENGTH = cfg["CALC_LENGTH"]
MARGIN = cfg["MARGIN"]
fourier_peak_threshold = cfg["fourier_peak_threshold"]

frame_width = FRAME_W
frame_height = FRAME_H

pixel_size_eff_x = pixel_size * sensor_width / frame_width
pixel_size_eff_y = pixel_size * sensor_height / frame_height

IFOV_x = 2 * np.degrees(np.arctan(pixel_size_eff_x / (2 * focal_length)))
IFOV_y = 2 * np.degrees(np.arctan(pixel_size_eff_y / (2 * focal_length)))

# ===== Global buffers =====
WI, HE = frame_width // BOX_SIZE + 1, frame_height // BOX_SIZE + 1
fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
freq_counter = np.zeros((HE, WI))
pixel_count_buff = np.zeros(CALC_LENGTH)

# ===== Global variables =====
mode = 1
roi_mode1 = None
MARGIN3_factor = 1
LED_CIRCLE_SIZE = 0.05
bbox_mode2 = None
flicker_buff = None
scounter = tcounter = m2counter = 0
top_point, bottom_point = None, None
pixel_buffer = None
frame_idx = 0
flicker_stable_count = 0
STABLE_FRAMES_THRESHOLD = 5
bbox_mode3 = None
ROI_STABLE_FRAMES = 1
ROI_HISTORY = 10
roi_history = []
failure_count_m3 = 0
MAX_FAILURES_M3 = 40
roi_stable_count = 0

# ===== MODE 4 VARIABLES =====
bbox_mode4 = None                    # ROI for focused scanning in Mode 4
mode4_frame_counter = 0              # Counter for model re-validation
mode4_fft_failure_count = 0          # Counter for FFT frequency failures
MAX_FFT_FAILURES_MODE4 = 20          # Max failures before returning to Mode 1
last_prediction_result = None        # Store last prediction (True/False)
false_alarm_start_time = None        # Timer for FALSE ALARM display

# ===== VIDEO RECORDING VARIABLES =====
is_recording = False
video_writer = None
recording_filename = ""
frame_count_record = 0
recording_start_time = None

# ===== Read FPS from /etc/reucamd.conf =====
fps_conf = configparser.ConfigParser()
fps_conf.read("/etc/reucamd.conf")
FPS = fps_conf.getint("camera", "framerate", fallback=cfg["FPS"])

cross_conf_path = os.path.join(os.path.dirname(__file__), '..', 'etc', 'reucamd.conf')
PIXEL_HISTORY = FPS * 2

# ===== AI MODEL PREDICTION FUNCTION (TFLite INT8) =====
def predict_detector(frame_rgb, bbox):
    """Run TFLite INT8 model on ROI. Returns (is_detected, confidence)"""
    if tflite_interpreter is None:
        return True, 1.0  # Auto-confirm if no model
    
    x, y, w, h = [int(v) for v in bbox]
    x, y = max(0, x), max(0, y)
    x2, y2 = min(frame_rgb.shape[1], x + w), min(frame_rgb.shape[0], y + h)
    
    if x2 <= x or y2 <= y:
        return False, 0.0
    
    roi = frame_rgb[y:y2, x:x2]
    roi_resized = cv2.resize(roi, (MODEL_INPUT_SIZE, MODEL_INPUT_SIZE))
    roi_input = np.expand_dims(roi_resized, axis=0).astype(np.uint8)  # INT8 model expects uint8
    
    tflite_interpreter.set_tensor(input_details[0]['index'], roi_input)
    tflite_interpreter.invoke()
    prediction = tflite_interpreter.get_tensor(output_details[0]['index'])[0][0]
    
    is_detected = prediction > MODEL_CONFIDENCE_THRESHOLD
    return is_detected, float(prediction)

# ===== VIDEO RECORDING FUNCTIONS =====
def start_recording():
    global is_recording, video_writer, recording_filename, frame_count_record, recording_start_time
    if is_recording:
        logging.info("Already recording!")
        return
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    recording_filename = f"RAW_camera_recording_{timestamp}.mp4"
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(recording_filename, fourcc, FPS, (FRAME_W, FRAME_H))
    if not video_writer.isOpened():
        logging.error("Failed to initialize RAW video writer")
        return
    is_recording = True
    recording_start_time = time.time()
    frame_count_record = 0
    logging.info(f"RAW Recording started: {recording_filename}")
    print(f"RAW_RECORDING_STARTED: {recording_filename}")

def stop_recording():
    global is_recording, video_writer, recording_filename, frame_count_record
    if not is_recording:
        logging.info("Not recording!")
        return
    if video_writer:
        video_writer.release()
        video_writer = None
    recording_duration = time.time() - recording_start_time if recording_start_time else 0
    is_recording = False
    logging.info(f"RAW Recording stopped. Saved: {recording_filename} ({frame_count_record} frames, {recording_duration:.1f}s)")
    print(f"RAW_RECORDING_STOPPED: {recording_filename}, frames: {frame_count_record}, duration: {recording_duration:.1f}s")

def record_raw_frame(yuv_data):
    global frame_count_record
    if is_recording and video_writer and yuv_data is not None:
        raw_frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
        video_writer.write(raw_frame)
        frame_count_record += 1

# ===== REUCAM SERVICE =====
class ReucamClient(ReuCameraClient):
    def __init__(self, svc):
        super().__init__()
        self.svc = svc

    def cmd_onFrame(self, cmd):
        frame = cmd.frame
        self.svc.onFrame(frame)

class ReucamService(threading.Thread):
    def __init__(self):
        super().__init__()
        self.daemon = True
        self.client = None
        self.lock = threading.Lock()
        self.I = None
        self.ts = None
        self.frame_event = threading.Event()
        self.bActive = False

    def run(self):
        self.bActive = True
        logging.info("ReucamService: started")
        backoff = 1.0
        while self.bActive:
            client = None
            try:
                client = ReucamClient(self)
                self.client = client
                client.connect(ReuCameraService.SERVER_ADDR)
                logging.info("ReucamService: connected")
                client.cmd_StartStream(3)
                client.recv_loop()
            except Exception:
                logging.exception("ReucamService error - reconnecting")
                try:
                    if client:
                        client.close()
                except Exception:
                    pass
                self.client = None
                time.sleep(backoff)
                backoff = min(backoff * 2, 30.0)
                continue
            else:
                backoff = 1.0
                try:
                    if client:
                        client.close()
                except Exception:
                    pass
                self.client = None
            finally:
                time.sleep(0.2)

    def onFrame(self, frame):
        try:
            if not frame:
                return
            if getattr(frame, 'is_raw', False) and len(frame.data) == FRAME_W * FRAME_H * 3 // 2:
                with self.lock:
                    self.I = frame.data
                    self.ts = frame.timestamp
                    self.frame_event.set()
        except Exception:
            logging.exception("Error in onFrame handler")

    def onFireBorder(self, param1, param2):
        try:
            if self.client is None:
                return
            self.client.cmd_Notify(REUCAM_NOTIFY_FIREBORDER, param1, param2)
        except Exception:
            pass

# ===== AUXILIARY FUNCTIONS =====
def convert_yuv_to_bgr(yuv_data, width, height):
    y_size = width * height
    uv_size = y_size // 4
    y = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((height, width))
    u = np.frombuffer(yuv_data[y_size:y_size+uv_size], dtype=np.uint8).reshape((height//2, width//2))
    v = np.frombuffer(yuv_data[y_size+uv_size:y_size+2*uv_size], dtype=np.uint8).reshape((height//2, width//2))
    u_up = cv2.resize(u, (width, height), interpolation=cv2.INTER_LINEAR)
    v_up = cv2.resize(v, (width, height), interpolation=cv2.INTER_LINEAR)
    yuv = cv2.merge((y, u_up, v_up))
    return cv2.cvtColor(yuv, cv2.COLOR_YUV2BGR)

def get_fft_frequency(sig, fps):
    sig = sig - np.mean(sig)
    fft_vals = np.abs(np.fft.rfft(sig))[0:]
    f_axis = np.linspace(0, fps / 2, len(fft_vals) + 1)[1:]
    if len(fft_vals) == 0:
        return 0
    f_min, f_max = LED_FREQ - FREQ_TOL, LED_FREQ + FREQ_TOL
    f_min_R, f_max_R = LED_FREQ - FREQ_RANGE, LED_FREQ + FREQ_RANGE
    mask_freq = (f_axis >= f_min) & (f_axis <= f_max)
    mask_range = (f_axis >= f_min_R) & (f_axis <= f_max_R)
    mask_outer_only = mask_range & (~mask_freq)
    if len(fft_vals[mask_freq]) == 0 or len(fft_vals[mask_outer_only]) == 0:
        return 0
    inner_sum = np.sum(fft_vals[mask_freq]) / len(fft_vals[mask_freq])
    outer_only_sum = np.sum(fft_vals[mask_outer_only]) / len(fft_vals[mask_outer_only])
    return LED_FREQ if 0.2 * inner_sum > outer_only_sum else 0

def process_box(frame, gray, B2, h, w):
    b2 = B2[h*BOX_SIZE:(h+1)*BOX_SIZE, w*BOX_SIZE:(w+1)*BOX_SIZE]
    z = np.roll(fourier_buff[:, h, w], 1)
    z[0] = np.sum(b2)
    fourier_buff[:, h, w] = z
    freq = get_fft_frequency(fourier_buff[:, h, w], FPS)
    if abs(freq - LED_FREQ) < FREQ_TOL:
        freq_counter[h, w] += 1
        x1, y1 = w * BOX_SIZE, h * BOX_SIZE
        x2 = min((w + 1) * BOX_SIZE, frame_width)
        y2 = min((h + 1) * BOX_SIZE, frame_height)
        return (x1, y1, x2, y2), freq
    freq_counter[h, w] = 0
    return None, None

def process_flicker_pixels(frame, roi, flicker_buff, frame_count, buffer_len=10, thresh=60):
    global flicker_stable_count, mode, bbox_mode3
    if roi is None:
        return None, flicker_buff, frame_count, bbox_mode3
    x1, y1, x2, y2 = roi
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(frame.shape[1]-1, x2), min(frame.shape[0]-1, y2)
    if x2 <= x1 or y2 <= y1:
        return None, flicker_buff, frame_count, bbox_mode3
    roi_mode2 = frame[y1:y2, x1:x2]
    roi_gray = cv2.cvtColor(roi_mode2, cv2.COLOR_BGR2GRAY)
    h, w = roi_gray.shape
    if flicker_buff is None or flicker_buff.shape[1:] != (h, w):
        flicker_buff = np.zeros((buffer_len, h, w), dtype=np.uint8)
        frame_count = 0
        flicker_stable_count = 0
    flicker_buff = np.roll(flicker_buff, -1, axis=0)
    flicker_buff[-1] = roi_gray
    frame_count += 1
    mask = np.zeros((h, w), dtype=np.uint8)
    if frame_count > buffer_len:
        diff = flicker_buff.max(axis=0) - flicker_buff.min(axis=0)
        mask[diff > thresh] = 200
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        if contours:
            all_points = np.vstack(contours)
            x, y, w_box, h_box = cv2.boundingRect(all_points)
            MARGIN3 = 25
            x_margin = max(0, x1 + x - MARGIN3)
            y_margin = max(0, y1 + y - MARGIN3)
            x2_margin = min(frame.shape[1], x1 + x + w_box + MARGIN3)
            y2_margin = min(frame.shape[0], y1 + y + h_box + MARGIN3)
            cv2.rectangle(frame, (x1+x, y1+y), (x1+x+w_box, y1+y+h_box), (0, 0, 255), 2)
            flicker_stable_count += 1
            if flicker_stable_count >= STABLE_FRAMES_THRESHOLD:
                bbox_mode3 = (x_margin, y_margin, x2_margin - x_margin, y2_margin - y_margin)
                logging.info(f"bbox_mode3={bbox_mode3}")
                mode = 3
        else:
            flicker_stable_count = 0
            bbox_mode3 = None
            mode = 1
    return mask, flicker_buff, frame_count, bbox_mode3

def convert_bboxes_to_reucam_params(bbox):
    x0 = math.ceil(bbox[0] * FRAME_W / frame_width)
    x1 = math.ceil(bbox[2] * FRAME_W / frame_width)
    y0 = math.ceil(bbox[1] * FRAME_H / frame_height)
    y1 = math.ceil(bbox[3] * FRAME_H / frame_height)
    w, h = x1 - x0, y1 - y0
    if frame_width > 640:
        w, h = math.ceil(w / 2), math.ceil(h / 2)
    x0 = max(0, min(65535, x0))
    y0 = max(0, min(65535, y0))
    w, h = max(0, min(255, w)), max(0, min(255, h))
    param1 = x0
    try:
        param2 = struct.pack('HBB', y0, w, h)
    except struct.error:
        param2 = struct.pack('HBB', 0, 0, 0)
    param2 = struct.unpack('i', param2)[0]
    return param1, param2

def calculate_alignment(bbox, center_x0, center_y0):
    """Calculate alignment angles from bbox center to frame center"""
    x, y, w, h = [int(v) for v in bbox]
    center_x = x + w // 2
    center_y = y + h // 2
    ANG_THRESH = 0.2
    y_alignment_deg = -2 * np.round((center_x - center_x0) * IFOV_x, 1)
    y_alignment_deg = 0 if abs(y_alignment_deg) < ANG_THRESH else y_alignment_deg
    x_alignment_deg = -2 * np.round((center_y - center_y0) * IFOV_y, 1)
    x_alignment_deg = 0 if abs(x_alignment_deg) < ANG_THRESH else x_alignment_deg
    return x_alignment_deg, y_alignment_deg, center_x, center_y

def reset_to_mode1():
    """Reset all state variables and return to Mode 1"""
    global mode, bbox_mode3, bbox_mode4, flicker_stable_count, roi_stable_count
    global fourier_buff, flicker_buff, mode4_frame_counter, mode4_fft_failure_count
    global failure_count_m3, tcounter, pixel_count_buff, last_prediction_result
    mode = 1
    bbox_mode3 = None
    bbox_mode4 = None
    flicker_stable_count = 0
    roi_stable_count = 0
    failure_count_m3 = 0
    tcounter = 0
    mode4_frame_counter = 0
    mode4_fft_failure_count = 0
    last_prediction_result = None
    pixel_count_buff[:] = 0
    fourier_buff = np.zeros((CALC_LENGTH, HE, WI))
    flicker_buff = None
    logging.info("Reset to Mode 1")

# Set stdin to non-blocking
fd = sys.stdin.fileno()
fl = fcntl.fcntl(fd, fcntl.F_GETFL)
fcntl.fcntl(fd, fcntl.F_SETFL, fl | os.O_NONBLOCK)

def check_stdin():
    try:
        line = sys.stdin.readline()
        if line:
            return line.strip()
    except Exception:
        return None
    return None

last_fireborder = None
last_fire_time = 0

# ===== MAIN LOOP =====
if __name__ == "__main__":
    reucam = ReucamService()
    reucam.start()
    
    def cleanup():
        try:
            if is_recording:
                stop_recording()
            reucam.onFireBorder(0, 0)
        except Exception:
            pass
    
    atexit.register(cleanup)
    
    def handle_signal(signum, frame):
        cleanup()
        sys.exit(0)
    
    signal.signal(signal.SIGINT, handle_signal)
    signal.signal(signal.SIGTERM, handle_signal)
    signal.signal(signal.SIGPIPE, signal.SIG_DFL)

    I = None
    frame_count = 0
    fps_start_time = time.time()
    current_fps = FPS
    
    while True:
        if reucam.frame_event.wait(1):
            break
        logging.info("Waiting for camera to connect...")

    print("=== LED DETECTOR WITH AI MODEL (TFLite INT8) ===")
    print("Commands: 's'=start recording, 'p'=stop recording")
    print("=================================================")
    
    while True:
        line = check_stdin()
        if line == "s":
            start_recording()
        elif line == "p":
            stop_recording()
        elif line == "RESET_BORDER":
            logging.info("Received RESET_BORDER command")
            reucam.onFireBorder(0, 0)
            reset_to_mode1()

        reucam.frame_event.clear()
        if reucam.I == I:
            continue
        I = reucam.I
        ts = reucam.ts

        # FPS calculation
        frame_count += 1
        current_time = time.time()
        if current_time - fps_start_time >= 1.0:
            current_fps = frame_count / (current_time - fps_start_time)
            print(f"FPS: {current_fps:.1f} | Mode: {mode}", flush=True)
            frame_count = 0
            fps_start_time = current_time
            FPS = current_fps

        if I is None:
            continue

        yuv_data = reucam.I
        record_raw_frame(yuv_data)

        # Extract grayscale and create display frame
        y_size = FRAME_W * FRAME_H
        gray = np.frombuffer(yuv_data[0:y_size], dtype=np.uint8).reshape((FRAME_H, FRAME_W))
        _, B2 = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY)
        frame = cv2.cvtColor(gray, cv2.COLOR_GRAY2BGR)
        frame[10:20, 260:380, :] = 0  # Delete clock overlay

        # Read cross position for alignment
        cross_conf = configparser.ConfigParser()
        cross_conf.read(cross_conf_path)
        center_x0 = cross_conf.getint("camera", "cross_w_pos", fallback=frame_width // 2)
        center_y0 = cross_conf.getint("camera", "cross_h_pos", fallback=frame_height // 2)

        # ===== MODE 1: Full FFT Scan =====
        if mode == 1:
            print("1 0 0", flush=True)
            found_boxes = []
            for h in range(HE):
                for w in range(WI):
                    cv2.rectangle(frame, (w*BOX_SIZE, h*BOX_SIZE), 
                                  (w*BOX_SIZE+BOX_SIZE, h*BOX_SIZE+BOX_SIZE), (255, 255, 255), 1)
                    box, freq = process_box(frame, gray, B2, h, w)
                    if box is not None:
                        found_boxes.append(box)

            if found_boxes:
                x1 = max(0, min(b[0] for b in found_boxes) - MARGIN)
                y1 = max(0, min(b[1] for b in found_boxes) - MARGIN)
                x2 = min(frame_width, max(b[2] for b in found_boxes) + MARGIN)
                y2 = min(frame_height, max(b[3] for b in found_boxes) + MARGIN)
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
                roi_stable_count += 1
                if roi_stable_count >= ROI_STABLE_FRAMES:
                    bbox_mode2 = (x1 - MARGIN, y1 - MARGIN, x2 + MARGIN, y2 + MARGIN)
                    mode = 2
                    logging.info("Mode 1 -> Mode 2")
            else:
                roi_stable_count = 0

        # ===== MODE 2: Refine Bbox with Flicker Analysis =====
        elif mode == 2:
            frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
            frame[10:20, 260:380, :] = 0
            print("1 0 0", flush=True)
            if bbox_mode2 is not None:
                mask, flicker_buff, frame_idx, bbox_mode3 = process_flicker_pixels(
                    frame, bbox_mode2, flicker_buff, frame_idx)

        # ===== MODE 3: AI Model Prediction =====
        elif mode == 3:
            frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
            frame[10:20, 260:380, :] = 0
            
            if bbox_mode3 is not None:
                x, y, w, h = [int(v) for v in bbox_mode3]
                
                # Run AI model prediction
                is_detected, confidence = predict_detector(frame, bbox_mode3)
                logging.info(f"Model prediction: detected={is_detected}, confidence={confidence:.3f}")
                
                if is_detected:
                    # POSITIVE: Transition to Mode 4
                    bbox_mode4 = (max(0, x - MARGIN_MODE4), 
                                  max(0, y - MARGIN_MODE4),
                                  min(w + 2*MARGIN_MODE4, frame_width - x + MARGIN_MODE4),
                                  min(h + 2*MARGIN_MODE4, frame_height - y + MARGIN_MODE4))
                    mode4_frame_counter = 0
                    mode4_fft_failure_count = 0
                    last_prediction_result = True
                    mode = 4
                    logging.info(f"Mode 3 -> Mode 4 (CONFIRMED) bbox_mode4={bbox_mode4}")
                    
                    # Calculate and output alignment
                    x_deg, y_deg, cx, cy = calculate_alignment(bbox_mode3, center_x0, center_y0)
                    print(f"0 {x_deg} {y_deg}", flush=True)
                    
                    # Display CONFIRMED
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 3)
                    cv2.putText(frame, "CONFIRMED!", (x, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
                else:
                    # NEGATIVE: FALSE ALARM
                    false_alarm_start_time = time.time()
                    last_prediction_result = False
                    logging.info("Mode 3 -> FALSE ALARM -> Mode 1")
                    
                    # Display FALSE ALARM
                    cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 0, 255), 3)
                    cv2.putText(frame, "FALSE ALARM!", (x, y - 10), 
                                cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)
                    print("2 0 0", flush=True)
                    reset_to_mode1()
            else:
                print("1 0 0", flush=True)
                reset_to_mode1()

        # ===== MODE 4: Focused FFT + Periodic Model Validation =====
        elif mode == 4:
            frame = convert_yuv_to_bgr(yuv_data, FRAME_W, FRAME_H)
            frame[10:20, 260:380, :] = 0
            
            if bbox_mode4 is not None:
                x, y, w, h = [int(v) for v in bbox_mode4]
                x, y = max(0, x), max(0, y)
                x2, y2 = min(frame_width, x + w), min(frame_height, y + h)
                
                # FFT check on ROI
                roi_binary = B2[y:y2, x:x2]
                non_zero_pixel_count = np.sum(roi_binary)
                pixel_count_buff = np.roll(pixel_count_buff, -1)
                pixel_count_buff[-1] = non_zero_pixel_count
                tcounter += 1
                
                fft_valid = True
                if tcounter > CALC_LENGTH:
                    detected_freq = get_fft_frequency(pixel_count_buff, FPS)
                    if abs(detected_freq - LED_FREQ) > FREQ_TOL:
                        mode4_fft_failure_count += 1
                        if mode4_fft_failure_count >= MAX_FFT_FAILURES_MODE4:
                            logging.info("Mode 4: FFT frequency lost -> Mode 1")
                            print("2 0 0", flush=True)
                            reset_to_mode1()
                            fft_valid = False
                    else:
                        mode4_fft_failure_count = 0
                
                if fft_valid and mode == 4:
                    # Periodic model re-validation
                    mode4_frame_counter += 1
                    if mode4_frame_counter >= MODEL_VALIDATION_INTERVAL:
                        mode4_frame_counter = 0
                        is_detected, confidence = predict_detector(frame, bbox_mode4)
                        logging.info(f"Mode 4 re-validation: detected={is_detected}, conf={confidence:.3f}")
                        if not is_detected:
                            logging.info("Mode 4: Model validation failed -> Mode 1")
                            print("2 0 0", flush=True)
                            reset_to_mode1()
                    
                    if mode == 4:  # Still in mode 4
                        # Calculate and output alignment
                        x_deg, y_deg, cx, cy = calculate_alignment(bbox_mode4, center_x0, center_y0)
                        print(f"0 {x_deg} {y_deg}", flush=True)
                        
                        # Visual indication - LOCKED ON TARGET
                        color = (0, 255, 0) if abs(x_deg) <= 0.5 and abs(y_deg) <= 0.5 else (0, 165, 255)
                        cv2.rectangle(frame, (x, y), (x2, y2), color, 3)
                        cv2.putText(frame, "LOCKED ON TARGET", (x, y - 30), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, color, 2)
                        cv2.putText(frame, f"Align: ({x_deg}, {y_deg})", (x, y - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
                        
                        # Send FireBorder notification
                        bbox_draw = [x, y, x2, y2]
                        param1, param2 = convert_bboxes_to_reucam_params(bbox_draw)
                        current_time = time.time()
                        if last_fireborder != (param1, param2) or current_time - last_fire_time > 0.05:
                            reucam.onFireBorder(0, 0)
                            time.sleep(0.01)
                            reucam.onFireBorder(param1, param2)
                            last_fireborder = (param1, param2)
                            last_fire_time = current_time
            else:
                print("1 0 0", flush=True)
                reset_to_mode1()

        else:
            print("1 0 0", flush=True)

        # Display mode and recording status
        mode_colors = {1: (255, 255, 255), 2: (0, 255, 255), 3: (255, 0, 255), 4: (0, 255, 0)}
        mode_names = {1: "SCANNING", 2: "REFINING", 3: "PREDICTING", 4: "LOCKED"}
        cv2.putText(frame, f"Mode {mode}: {mode_names.get(mode, '')}", (40, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.2, mode_colors.get(mode, (255,255,255)), 2)
        cv2.putText(frame, f"FPS: {current_fps:.1f}", (40, 110),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        if is_recording:
            cv2.putText(frame, f"REC: {frame_count_record} frames", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord('q'):
            break

    cv2.destroyAllWindows()